In [ ]:
import sys
import site
import os
from pathlib import Path

# Ensure Python uses the FiberSim environment
print("Python is using this environment:", sys.executable)

In [ ]:
# Get the directory of the Jupyter notebook
notebook_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()

# Search for FiberSim within common directory structures-the correct end folder is the second FiberPy
potential_paths = [
    notebook_dir / "Code" / "FiberPy" / "FiberPy",  # If running in a cloned repo
    Path.home() / "GitHub" / "FiberSim" / "Code" / "FiberPy" / "FiberPy"  # If installed in a GitHub directory
]

fiberpy_path = None
for path in potential_paths:
    if path.exists():
        fiberpy_path = path
        break

# If no valid path is found, ask the user
if fiberpy_path is None:
    fiberpy_path = input("Enter the path to the FiberPy directory: ")
    fiberpy_path = Path(fiberpy_path).resolve()

# Set the working directory
os.chdir(fiberpy_path)
print(f"Working directory set to: {os.getcwd()}")

In [ ]:
# Import fiberpy functions
from package.modules.characterize.characterize_model import characterize_model as characterize
from package.modules.fitting.fit_model import fit_model 
from package.modules.sample.sample_model import sample_model 
from package.modules.visualization.render import generate_images

print("FiberSim modules loaded successfully!")

In [ ]:
# Get the current working directory (FiberSim/Code/FiberPy/FiberPy/)
current_dir = Path.cwd()

# Navigate up 3 levels to reach FiberSim/
fiber_sim_root = current_dir.parents[2]

# Get the demo_files directory
demo_files_dir = fiber_sim_root / "demo_files"

print(f"Demo files directory: {demo_files_dir}")

# Dictionaries and Definitions

In [ ]:
# Define categories and their corresponding subdirectories
demo_categories = {
    "single_trials": ["isometric_activation", "k_tr"],
    "pca_curves": ["single_curve", "different_lengths"],
    "length_tension": ["steady_state"],
    "isotonic": ["force_velocity"],
    "model_comparison": ["parameter_adjustments"],
    "myofibrils": ["hs_with_sec", "multiple_hs_with_sec", "variable_hs", "mybpc", "sequential_hs", "spocs"],
    "isotypes": ["k_tr", "twitch", "specific_molecules", "switching"],
    "electrical_stimulation":["twitch"," semi_fused_tetanus", "afterloads"],
    "sampling": ["latin_hypercube"],
    "length_changes":["steps"],
    "validation":["force_balance"],
    "afterload":["afterload_twitch"],
    "mybpc":["load_shortening"]
}

# Automatically build demos_dict using loops
demos_dict = {
    f"{category}_{demo}": demo_files_dir / category / demo / "base" / "setup.json"
    for category, demos in demo_categories.items()
    for demo in demos
}

# # Print to check structure
print(demos_dict)
# print(list(demos_dict.keys()))

In [ ]:
# Function to run a demo
def run_fibersim_demo(demo_name):
    """
    Runs the appropriate FiberSim function based on the folder name.
    
    Parameters:
    demo_name (str): The name of the demo to run, as defined in demos_dict.
    """
    if demo_name not in demos_dict:
        print(f"Error: Demo '{demo_name}' not found.")
        return

    demo_file = demos_dict[demo_name]

    if not demo_file.exists():
        print(f"Error: Demo file '{demo_file}' not found.")
        return

    # Extract the directory name right after `demo_base_dir`
    parent_folder = demo_file.relative_to(demo_files_dir).parts[0]

    # Determine which function to run based on folder structure
    if parent_folder == "fitting": # this does not currently work
        print(f"Running fitting demo: {demo_name}")
        fit_model(str(demo_file))
    elif parent_folder == "sampling":
        print(f"Running sampling demo: {demo_name}")
        sample_model(str(demo_file))
    elif parent_folder == "visualization": # this does not currently work
        print(f"Running visualization demo: {demo_name}")
        generate_images(str(demo_file))
    else:
        print(f"Running characterization demo: {demo_name}")
        characterize(str(demo_file))

# Demos

## Single Trials

### Isometric activation

Runs a single simulation in which a half-sarcomere is held isometric and activated in a solution with a pCa of 4.5

In [ ]:
run_fibersim_demo("single_trials_isometric_activation")

### k_tr trial

Runs a single simulation in which a half-sarcomere is activated in pCa 4.5 and then subjected to a k_tr maneuver

In [ ]:
run_fibersim_demo("single_trials_k_tr")


## pCa curves

### Single curve

Builds on the single k_tr trial demo and runs simulations in which a half-sarcomere is subjected to a ktr maneuver at a range of pCa values

In [ ]:
run_fibersim_demo("pca_curves_single_curve")

### Different lengths

Builds on the single pCa curve trial and runs simulations in which a half-sarcomere is held at different lengths and subjected to a ktr maneuver at a range of pCa values

In [ ]:
run_fibersim_demo("pca_curves_different_lengths")

## Length-tension

### Steady-state

+ Runs a set of simulations
+ Each simulation calculates the properties of a half-sarcomere held at a different length
+ In each simulation, the fiber starts relaxed but is then maximally activated
+ Once the simulations have finished, the data are analyzed to determine the passive, active, and total length-tension curves

In [ ]:
run_fibersim_demo("length_tension_steady_state")

## Isotonic

### Force velocity

+ Runs a simulation in which a half-sarcomere is activated in pCa 4.5 and held isometric
  + This determines the isometric force
+ The code then runs a set of simulations in which the half-sarcomere is activated in pCa 4.5 and, once force has reached steady-state, allowed to shorten against different fractions of the isometric force.
+ Force-velocity and force-power curves are fitted to the simulation data.

In [ ]:
run_fibersim_demo("isotonic_force_velocity")

## Model comparison

### Parameter adjustments

+ Builds on the single pCa curve demo and runs simulations in which a half-sarcomere is subjected to a ktr maneuver at a range of pCa values
+ Uses a manipulations structure to create 4 models with different parameter values
+ Plots summaries of the simulation

In [ ]:
run_fibersim_demo("model_comparison_parameter_adjustments")

## Myofibrils

### Half-sarcomere with series compliance

Runs a single simulation in which a single half-sarcomere connected in series with a linear spring is activated and deactivated by step changes in the Ca2+ concentration

In [ ]:
run_fibersim_demo("myofibrils_hs_with_sec")

### Multiple half-sarcomere with series compliance
Runs a single simulation in which a myofibril composed of 10 half-srcomeres connected in series with a linear spring is activated and deactivated by step changes in the Ca<sup>2+</sup> concentration

In [ ]:
run_fibersim_demo("myofibrils_multiple_hs_with_sec")

### Variable half-sarcomeres

Builds on the multiple half-sarcomeres with series compliance demo but reduces the attachment rate for myosin in a half-sarcomere in the middle of the myofibril

In [ ]:
run_fibersim_demo("myofibrils_variable_hs")

### mybpc ***

In [ ]:
run_fibersim_demo("myofibrils_mybpc")

### sequential_hs ***

In [ ]:
run_fibersim_demo("myofibrils_sequential_hs")

### spocs ***

In [ ]:
run_fibersim_demo("myofibrils_spocs")

## Isotypes

### k_tr
+ Simulates a half-sarcomere activated in solutions with 5 different pCa values and subject to a ktr maneuver.
+ Plots a summary of the simulation.
+ Plots the number of molecules in each state against time for each isotype.

In [ ]:
run_fibersim_demo("isotypes_k_tr")

### Twitch
This demo simulates how changing the relative proportions of two isotypes of myosin influences a twitch contraction.

In [ ]:
run_fibersim_demo("isotypes_twitch")

### Specific molecules
+ Simulates a half-sarcomere undergoing an isometric contraction
+ There are two isotypes of myosin. The kinetics for the second isotype are slower than that of the first isotype
+ The first head in every myosin dimer is isotype 1. The second head in every dimer is isotype 2.
+ There are also two isotypes of MyBP-C. The first isotype attaches to actin. The second isotype can transition to a state where it stabilizes myosin heads.
+ There are 27 MyBP-C molecules arranged in 9 stripes of 3 molecules. All of the molecules within a stripe have the same isotype but the isotype alternates between stripes.
+ You can make the simulations smoother by increasing the m_n parameter in the setup file.
+ Status files are written for every 10th time-step. You can increase the time-resolution of the isotype plots by changing the time_steps parameter in the options file. For example, changing the string to 1:3:500 would save the status at every 3 time-step.

In [ ]:
run_fibersim_demo("isotypes_specific_molecules")

### Switching ***

In [ ]:
run_fibersim_demo("isotypes_switching")

## Electrical stimulation

### Twitch
Runs a single simulation in which a half-sarcomere is held isometric and activated by a Ca<sup>2+</sup> transient

In [ ]:
run_fibersim_demo("electrical_stimulation_twitch")

### Semi-fused tetanus
Runs a single simulation in which a half-sarcomere is held isometric and activated by a sequenece of Ca<sup>2+</sup> transients.

In [ ]:
run_fibersim_demo("electrical_stimulation_ semi_fused_tetanus")

### Afterloads
Runs a set of simulations in which a half-sarcomere is activated by a Ca<sup>2+</sup> transient and allowed to shorten against a range of afterloads.

In [ ]:
run_fibersim_demo("electrical_stimulation_afterloads")

## Length changes

### Steps
+ Runs a set of simulations with step stretches of different sizes imposed at different activation levels

In [ ]:
run_fibersim_demo("length_changes_steps")

## Sampling

### Latin hypercube
+ Samples a base model by running 30 free-form simulations, each of which is based on a different model and simulates a protocol with a slightly different Ca2+ transient
+ After the simulations are complete, calls a custom-written Python function that
  + Deduces summary statistics for each simulation
  + Creates figures summarizing the results

In [ ]:
run_fibersim_demo("sampling_latin_hypercube")

## Validation

### Force-balance

+runs simulations for 6 fixed-end activations where the thick and thin filaments have different stiffnesses
+saves a status file for the last time-point for each simulation
+calls a demo-specific Python file that
  +loops through the simulations
  +calculates the unbalanced force at each node
  +saves a summary of the results for each simulation as a figure

In [ ]:
run_fibersim_demo("validation_force_balance")